Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [1]:
import retinoto_py as fovea
args = fovea.Params(do_mask=True)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=True, do_fovea=False, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=64, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=50, n_stop=19200, n_train_stop=19200, n_val_stop=6400, lr=0.0001, delta1=0.05, delta2=0.001, weight_decay=0.0, shuffle=True, verbose=True)

# transfer learning of each network on the  masked dataset

In [ ]:
%rm cached_data/20_model_name=resnet101_dataset=full_retrained.lock
model_name = args.model_name
for dataset in ['full']:# <HACK until bbox is finished> fovea.all_datasets:
    name = f'20_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    print(50*'.')

Training model resnet101, file= cached_data/20_model_name=resnet101_dataset=full_retrained.pth - image_size=224
Model has 1000 output features to final FC layer for 1000 classes.
Starting learning...


Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

epoch=1/50:   0%|          | 0/300 [00:00<?, ?it/s]

epoch=2/50:   0%|          | 0/300 [00:00<?, ?it/s]

epoch=3/50:   0%|          | 0/300 [00:00<?, ?it/s]

...

epoch=34/50:   0%|          | 0/300 [00:00<?, ?it/s]

epoch=35/50:   0%|          | 0/300 [00:00<?, ?it/s]

epoch=36/50:   0%|          | 0/300 [00:00<?, ?it/s]

epoch=37/50:   0%|          | 0/300 [00:00<?, ?it/s]

epoch=38/50:   0%|          | 0/300 [00:00<?, ?it/s]

## Model comparison

TODO Faire un nouveau notebook 23_

In [ ]:
results_13 = fovea.pd.read_json(args.data_cache / '13_model_comparison.json')
results_13

In [ ]:
for dataset in fovea.all_datasets:
    name = f'20_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    results = fovea.pd.read_json(json_filename)
    print(results)

## Plot learning evolution

In [ ]:
fig, ax = fovea.plt.subplots()
for dataset in fovea.all_datasets:
    color = 'r'
    lw = 1

    name = f'20_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)

    df_train = fovea.pd.read_json(json_filename, orient='records')

    df_train_roll = df_train.rolling(window=5, min_periods=1, center=False).mean()
    ax = df_train_roll.plot(x='total_image', y='acc_train', 
                        c=color, ls='dashed', lw=lw,
                        grid=True, ax=ax, label='TRAIN: ' + args.model_name)    
    ax = df_train_roll.plot(x='total_image', y='acc_val', 
                        c=color, lw=lw,
                        grid=True, ax=ax, label='VAL: ' + args.model_name)   

In [ ]:
df_train